#Case Eleflow

Todo esse código foi construido e executado no DataBricks Comunity Edition.

Primeiro passo foi fazer o upload dos arquivos nas pastas AIR_CIA e VRA que eu criei, abaixo vou listar o contéudo de cada uma dessas pastas.

Obs: Essa etapa foi manual, uma possível melhoria seria fazer isso de forma automatizada.

In [0]:
%fs ls "dbfs:/FileStore/shared_uploads/AIR_CIA"


path,name,size,modificationTime
dbfs:/FileStore/shared_uploads/AIR_CIA/ANAC_20211220_203627.csv,ANAC_20211220_203627.csv,4346,1658966567000
dbfs:/FileStore/shared_uploads/AIR_CIA/ANAC_20211220_203643.csv,ANAC_20211220_203643.csv,1863,1658966568000
dbfs:/FileStore/shared_uploads/AIR_CIA/ANAC_20211220_203733.csv,ANAC_20211220_203733.csv,754,1658966570000


In [0]:
%fs ls "dbfs:/FileStore/shared_uploads/VRA"

path,name,size,modificationTime
dbfs:/FileStore/shared_uploads/VRA/VRA_20211.json,VRA_20211.json,21457666,1658966401000
dbfs:/FileStore/shared_uploads/VRA/VRA_202110.json,VRA_202110.json,21756145,1658966405000
dbfs:/FileStore/shared_uploads/VRA/VRA_202111.json,VRA_202111.json,23454906,1658966409000
dbfs:/FileStore/shared_uploads/VRA/VRA_20212.json,VRA_20212.json,15995220,1658966414000
dbfs:/FileStore/shared_uploads/VRA/VRA_20213.json,VRA_20213.json,14460053,1658966416000
dbfs:/FileStore/shared_uploads/VRA/VRA_20214.json,VRA_20214.json,10664134,1658966417000
dbfs:/FileStore/shared_uploads/VRA/VRA_20215.json,VRA_20215.json,13098196,1658966419000
dbfs:/FileStore/shared_uploads/VRA/VRA_20216.json,VRA_20216.json,15024212,1658966421000
dbfs:/FileStore/shared_uploads/VRA/VRA_20217.json,VRA_20217.json,19565235,1658966423000
dbfs:/FileStore/shared_uploads/VRA/VRA_20218.json,VRA_20218.json,20120227,1658966425000


Importando as funções que irei utilizar nesse código

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Row
from unicodedata import normalize
import requests

Para normalizar os cabeçalhos das tabelas, utilizei duas funções uma para tirar palavras do CamelCase e outra para retirar a acentuação das palavras e retirar o espaço.

Fiz isso em função pois esse procedimento foi feito mais de uma vez, então para reaproveitar o código.

OBS: Algumas melhorias que poderiam ser feitas aqui são, em casos que temos siglas em maiusculo a função do change_case colocará _ antes e após cada letra e isto não é o desejado. Como no nosso caso só aparece as siglas ICAO, IATA e CNPJ substitui esses casos na função, mas poderia ser criada uma forma de substituir para toda sigla.

Também retirei a acentuação das palavras do cabeçalho pois facilita a manipulação do sql

In [0]:
def change_case(word:str):
    """Tirando a uma palavra do CamelCase para o snake_case"""
    return ''.join(['_'+i.lower() if i.isupper() else i for i in word]).lstrip('_')

def normalize_header(header:list):
    """Função para normalizar os cabeçalhos dos dataframes"""
    new_header = []
    for column in header:
        #tirando a acentuação
        column = normalize('NFKD', column).encode('ASCII','ignore').decode('ASCII')
        column = change_case(column)
        column = column.replace('i_c_a_o', 'icao')
        column = column.replace('i_a_t_a', 'iata')
        column = column.replace('c_n_p_j', 'iata')
        column = column.replace(' ', '_')
        column = column.replace('__', '_')
        column = column.replace('-', '')
        new_header.append(column)
        
    return new_header

#Tabela AIR_CIA

Realizando a leitura da tabela a partir dos csv's que foram salvos na pasta AIR_CIA

In [0]:
air_cia = spark.read.format("csv").option("delimiter", ";").option("header", "true").load('dbfs:/FileStore/shared_uploads/AIR_CIA')
air_cia.show()

+--------------------+---------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+
|        Razão Social|ICAO IATA|              CNPJ|   Atividades Aéreas|       Endereço Sede|            Telefone|              E-Mail| Decisão Operacional|Data Decisão Operacional|Validade Operacional|
+--------------------+---------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+
|ABSA - AEROLINHAS...|   LTG M3|00.074.635/0001-33|TRANSPORTE AÉREO ...|AEROPORTO INTERNA...|      (11) 5582-8055|      gar@tam.com.br|       DECISÃO Nº 41|              22/04/2015|          23/04/2025|
|AEROSUL TÁXI AÉRE...|   ASO 2S|27.315.694/0001-02|SERVIÇOS AÉREOS P...|RODOVIA PR 218, K...|      (43) 3176-4030|operacoes@aerosul...|      DECISÃO Nº 282|              10/02/2021|       

Normalizando o cabeçalho da tabela com as funções criadas

In [0]:
air_cia = air_cia.toDF(*normalize_header(air_cia.columns))
air_cia.show()

+--------------------+---------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+
|        razao_social|icao_iata|              iata|   atividades_aereas|       endereco_sede|            telefone|              e_mail| decisao_operacional|data_decisao_operacional|validade_operacional|
+--------------------+---------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+
|ABSA - AEROLINHAS...|   LTG M3|00.074.635/0001-33|TRANSPORTE AÉREO ...|AEROPORTO INTERNA...|      (11) 5582-8055|      gar@tam.com.br|       DECISÃO Nº 41|              22/04/2015|          23/04/2025|
|AEROSUL TÁXI AÉRE...|   ASO 2S|27.315.694/0001-02|SERVIÇOS AÉREOS P...|RODOVIA PR 218, K...|      (43) 3176-4030|operacoes@aerosul...|      DECISÃO Nº 282|              10/02/2021|       

Mudando o tipo das colunas de string para data, isto não foi pedido mas achei interressante implementar para não ficar com todas as colunas como string.

In [0]:
air_cia = air_cia.withColumn("data_decisao_operacional",F.to_date("data_decisao_operacional","dd/MM/yyyy"))
air_cia = air_cia.withColumn("validade_operacional",F.to_date("validade_operacional","dd/MM/yyyy"))

Quebrando a coluna icao_iata em duas, porém fiquei em dúvidas se deveria excluir a coluna icao_iata depois ou não, por segurança achei melhor não excluir.

In [0]:
air_cia = air_cia.withColumn("icao", F.split(F.col("icao_iata"), " ").getItem(0))
air_cia = air_cia.withColumn("iata", F.split(F.col("icao_iata"), " ").getItem(1))

In [0]:
air_cia.show()

+--------------------+---------+----+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+----+
|        razao_social|icao_iata|iata|   atividades_aereas|       endereco_sede|            telefone|              e_mail| decisao_operacional|data_decisao_operacional|validade_operacional|icao|
+--------------------+---------+----+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+----+
|ABSA - AEROLINHAS...|   LTG M3|  M3|TRANSPORTE AÉREO ...|AEROPORTO INTERNA...|      (11) 5582-8055|      gar@tam.com.br|       DECISÃO Nº 41|              2015-04-22|          2025-04-23| LTG|
|AEROSUL TÁXI AÉRE...|   ASO 2S|  2S|SERVIÇOS AÉREOS P...|RODOVIA PR 218, K...|      (43) 3176-4030|operacoes@aerosul...|      DECISÃO Nº 282|              2021-02-10|                null| ASO|
|ASTA LINHAS AÉREA...|   SUL 0

Salvando a tabela com o AIR_CIA

In [0]:
air_cia.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("AIR_CIA")

#Tabela VRA
Realizando a leitura a partir dos JSON's na pasta VRA

In [0]:
vra = spark.read.format("json").load('dbfs:/FileStore/shared_uploads/VRA')
vra.show()

+-------------------+-------------------+-----------------+-------------------+---------------+--------------------+-------------------+----------------+---------+-------------------+-------------------+-----------+
|    ChegadaPrevista|        ChegadaReal|CódigoAutorização|CódigoJustificativa|CódigoTipoLinha|ICAOAeródromoDestino|ICAOAeródromoOrigem|ICAOEmpresaAérea|NúmeroVoo|    PartidaPrevista|        PartidaReal|SituaçãoVoo|
+-------------------+-------------------+-----------------+-------------------+---------------+--------------------+-------------------+----------------+---------+-------------------+-------------------+-----------+
|2021-11-12 08:30:00|2021-11-12 08:24:00|                0|                N/A|              X|                KORD|               SBGR|             UAL|     0844|2021-11-11 22:00:00|2021-11-11 22:14:00|  REALIZADO|
|2021-11-15 08:30:00|2021-11-15 08:05:00|                0|                N/A|              X|                KORD|               SBGR|

Normalizando o cabeçalho da tabela com as funções criadas

In [0]:
vra = vra.toDF(*normalize_header(vra.columns))
vra.show()

+-------------------+-------------------+------------------+--------------------+-----------------+----------------------+---------------------+------------------+----------+-------------------+-------------------+------------+
|   chegada_prevista|       chegada_real|codigo_autorizacao|codigo_justificativa|codigo_tipo_linha|icao_aerodromo_destino|icao_aerodromo_origem|icao_empresa_aerea|numero_voo|   partida_prevista|       partida_real|situacao_voo|
+-------------------+-------------------+------------------+--------------------+-----------------+----------------------+---------------------+------------------+----------+-------------------+-------------------+------------+
|2021-11-12 08:30:00|2021-11-12 08:24:00|                 0|                 N/A|                X|                  KORD|                 SBGR|               UAL|      0844|2021-11-11 22:00:00|2021-11-11 22:14:00|   REALIZADO|
|2021-11-15 08:30:00|2021-11-15 08:05:00|                 0|                 N/A|       

Mudando o tipo das colunas de string para timestamp, isto não foi pedido mas achei interressante implementar para não ficar com todas as colunas como string.

In [0]:
vra = vra.withColumn("chegada_prevista",F.to_timestamp("chegada_prevista","yyyy-MM-dd HH:mm:ss"))
vra = vra.withColumn("chegada_real",F.to_timestamp("chegada_real","yyyy-MM-dd HH:mm:ss"))
vra = vra.withColumn("partida_prevista",F.to_timestamp("partida_prevista","yyyy-MM-dd HH:mm:ss"))
vra = vra.withColumn("partida_real",F.to_timestamp("partida_real","yyyy-MM-dd HH:mm:ss"))

In [0]:
vra.printSchema()

root
 |-- chegada_prevista: timestamp (nullable = true)
 |-- chegada_real: timestamp (nullable = true)
 |-- codigo_autorizacao: string (nullable = true)
 |-- codigo_justificativa: string (nullable = true)
 |-- codigo_tipo_linha: string (nullable = true)
 |-- icao_aerodromo_destino: string (nullable = true)
 |-- icao_aerodromo_origem: string (nullable = true)
 |-- icao_empresa_aerea: string (nullable = true)
 |-- numero_voo: string (nullable = true)
 |-- partida_prevista: timestamp (nullable = true)
 |-- partida_real: timestamp (nullable = true)
 |-- situacao_voo: string (nullable = true)



Verificando os valores da coluna codigo_autorizacao para saber se o tipo de dado é string ou number

In [0]:
vra.select("codigo_autorizacao").distinct().show()

+------------------+
|codigo_autorizacao|
+------------------+
|                 7|
|                 0|
|                 E|
|                 6|
|                 9|
|                 1|
|                 2|
|                 D|
|                 4|
|                 3|
+------------------+



substituindo 'N/A' por null na coluna codigo_justificativa

In [0]:
vra.na.replace('N/A', None,subset=["codigo_justificativa"]).show()

+-------------------+-------------------+------------------+--------------------+-----------------+----------------------+---------------------+------------------+----------+-------------------+-------------------+------------+
|   chegada_prevista|       chegada_real|codigo_autorizacao|codigo_justificativa|codigo_tipo_linha|icao_aerodromo_destino|icao_aerodromo_origem|icao_empresa_aerea|numero_voo|   partida_prevista|       partida_real|situacao_voo|
+-------------------+-------------------+------------------+--------------------+-----------------+----------------------+---------------------+------------------+----------+-------------------+-------------------+------------+
|2021-11-12 08:30:00|2021-11-12 08:24:00|                 0|                null|                X|                  KORD|                 SBGR|               UAL|      0844|2021-11-11 22:00:00|2021-11-11 22:14:00|   REALIZADO|
|2021-11-15 08:30:00|2021-11-15 08:05:00|                 0|                null|       

Salvando a tabela como VRA

In [0]:
vra.write.mode("overwrite").saveAsTable("VRA")

#Tabela Aerodromos

A fonte de dados dessa tabela é a [API](https://rapidapi.com/Active-api/api/airport-info/), para acessa-lá é necessário algumas informações que eu coletei e armazenei nas variavéis abaixo.

OBS: Uma possível melhoria seria deixar essas credênciais de acesso em variaveis de ambiente.

In [0]:
url='https://airport-info.p.rapidapi.com/airport'
headers={
'X-RapidAPI-Key': '1597c38be0mshc0944f214525361p1e09b7jsn98dc61d16c77',
'X-RapidAPI-Host': 'airport-info.p.rapidapi.com'
}

Um dos parametros de busca da API é o ICAO do aeroporto, para conseguir essa informações irei utilizar a tabela VRA que já foi salva acima.

In [0]:
data = spark.sql('select * from vra')

Coletando os aeroportos que devo buscar na api e deixando numa lista do Python. Estou buscando por todas os aeroportos que estão nas colunas icao_aerodromo_destino e icao_aerodromo_origem

In [0]:
aeroportos = data.select('icao_aerodromo_destino').union(data.select('icao_aerodromo_origem')).distinct().rdd.flatMap(lambda x: x).collect()

Criando a pasta aerodromos para receber os arquivos referente a saida da API

In [0]:
dbutils.fs.mkdirs("/FileStore/shared_uploads/aerodromos")

Acessando a API para cada aeroporto na lista e salvando o arquivo na pasta criada. Note que o formato está como txt, mas na realidade é um json. Tive problemas para salvar esse arquivo como a extensão json, mas consegui dessa forma.

In [0]:
for aeroporto in aeroportos:
    info = requests.get(url=url, headers=headers, params={'icao':{aeroporto}})
    try:
        if info.text=='{"error":{"text":"No airport found"}}\n':
            dbutils.fs.put("dbfs:/FileStore/shared_uploads/aerodromos/{}.txt".format(aeroporto),'{"iata":'+'"{}"'.format(aeroporto)+'}')
        else:
            dbutils.fs.put("dbfs:/FileStore/shared_uploads/aerodromos/{}.txt".format(aeroporto),info.text)
    except:
        pass

Listando todos os arquivos da pasta aerodromos

In [0]:
%fs ls "/FileStore/shared_uploads/aerodromos"

path,name,size,modificationTime
dbfs:/FileStore/shared_uploads/aerodromos/CYHM.txt,CYHM.txt,421,1659018874000
dbfs:/FileStore/shared_uploads/aerodromos/CYUL.txt,CYUL.txt,390,1659018880000
dbfs:/FileStore/shared_uploads/aerodromos/CYYZ.txt,CYYZ.txt,411,1659018797000
dbfs:/FileStore/shared_uploads/aerodromos/DGAA.txt,DGAA.txt,387,1659018857000
dbfs:/FileStore/shared_uploads/aerodromos/DIAP.txt,DIAP.txt,397,1659018908000
dbfs:/FileStore/shared_uploads/aerodromos/DNKN.txt,DNKN.txt,395,1659018896000
dbfs:/FileStore/shared_uploads/aerodromos/DNMM.txt,DNMM.txt,364,1659018791000
dbfs:/FileStore/shared_uploads/aerodromos/EBBR.txt,EBBR.txt,403,1659018841000
dbfs:/FileStore/shared_uploads/aerodromos/EBLG.txt,EBLG.txt,384,1659018790000
dbfs:/FileStore/shared_uploads/aerodromos/EBOS.txt,EBOS.txt,437,1659018877000


Realizando a leitura de todos os arquivos da pasta aerodromos

In [0]:
aeroportos = spark.read.format("json").option("inferSchema", True).load('dbfs:/FileStore/shared_uploads/aerodromos')

In [0]:
aeroportos.show()

+-----+--------+-----------+-----------------+----+----+----+----------+--------------------+----------+--------------------+----------------+-----------+--------------------+--------------------+-------------+----+--------------------+
| city| country|country_iso|           county|iata|icao|  id|  latitude|            location| longitude|                name|           phone|postal_code|               state|              street|street_number| uct|             website|
+-----+--------+-----------+-----------------+----+----+----+----------+--------------------+----------+--------------------+----------------+-----------+--------------------+--------------------+-------------+----+--------------------+
|Luque|Paraguay|         PY|                 | ASU|SGAS| 393|-25.241512|  Asunción, Paraguay|-57.514183|Silvio Pettirossi...|+595 21 688 2000|           |Departamento Central|Autopista Silvio ...|             |-180|http://www.dinac....|
|     |  Brazil|         BR|    Florianópolis| FLN|S

Normalizando o cabeçalho com a função que criei no inicio

In [0]:
aeroportos = aeroportos.toDF(*normalize_header(aeroportos.columns))
aeroportos.show()

+-----+--------+-----------+-----------------+----+----+----+----------+--------------------+----------+--------------------+----------------+-----------+--------------------+--------------------+-------------+----+--------------------+
| city| country|country_iso|           county|iata|icao|  id|  latitude|            location| longitude|                name|           phone|postal_code|               state|              street|street_number| uct|             website|
+-----+--------+-----------+-----------------+----+----+----+----------+--------------------+----------+--------------------+----------------+-----------+--------------------+--------------------+-------------+----+--------------------+
|Luque|Paraguay|         PY|                 | ASU|SGAS| 393|-25.241512|  Asunción, Paraguay|-57.514183|Silvio Pettirossi...|+595 21 688 2000|           |Departamento Central|Autopista Silvio ...|             |-180|http://www.dinac....|
|     |  Brazil|         BR|    Florianópolis| FLN|S

Salvando a tabela com o nome aerodromos

In [0]:
aeroportos.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("aerodromos")

#SQL

Para estas views, considerei como companhia mais atuante a companhia que fez mais decolagens a partir do aeroporto.

Considerei como rota, uma ligação entre dois aeroportos.

OBS: Nem todos os aeroportos possuem informações na API, como por exemplo, SBJD.

Para todas as analises só considerei voo com o status realizado.

#Criando a view seguindo

- Para cada companhia aérea trazer a rota mais utilizada com as seguintes informações:
    - Razão social da companhia aérea
    - Nome Aeroporto de Origem
    - ICAO do aeroporto de origem
    - Estado/UF do aeroporto de origem
    - Nome do Aeroporto de Destino
    - ICAO do Aeroporto de destino
    - Estado/UF do aeroporto de destino

In [0]:
%sql
create or replace view cia_aerea_rota_mais_usada as
select icao_empresa_aerea , razao_social ,icao_aerodromo_origem, origem.name as nome_aeroporto_origem, origem.location as localizacao_aeroporto_origem, icao_aerodromo_destino, destino.name as nome_aeroporto_destino, destino.location as localizacao_aeroporto_destino from air_cia
left join (
select icao_empresa_aerea ,icao_aerodromo_origem, icao_aerodromo_destino, count(*) as vezes, RANK() OVER (partition by icao_empresa_aerea  Order BY COUNT(*), icao_aerodromo_origem, icao_aerodromo_destino DESC) as ranking from vra
where situacao_voo='REALIZADO'
group by icao_empresa_aerea ,icao_aerodromo_origem, icao_aerodromo_destino) voo_rota on air_cia.icao=voo_rota.icao_empresa_aerea
left join aerodromos origem on origem.icao = voo_rota.icao_aerodromo_origem
left join aerodromos destino on destino.icao = voo_rota.icao_aerodromo_destino
where voo_rota.ranking=1



#Criando a view seguindo

- Para cada aeroporto trazer a companhia aérea com maior atuação no ano com as seguintes informações:
    - Nome do Aeroporto
    - ICAO do Aeroporto
    - Razão social da Companhia Aérea
    - Quantidade de Rotas à partir daquele aeroporto
    - Quantidade de Rotas com destino àquele aeroporto
    - Quantidade total de pousos e decolagens naquele aeroporto

In [0]:
%sql
create or replace view aeroportos_cia_mais_atuante as 
select aerodromos.name, aerodromos.icao, destino.rotas_destino, destino.pousos, origem.rotas_partida, origem.decolagens, empresa.razao_social as cia_mais_atuante from aerodromos
left join (select icao_aerodromo_destino, count(distinct(icao_aerodromo_origem)) as rotas_destino, count(icao_aerodromo_origem) as pousos from vra
where situacao_voo='REALIZADO'
group by icao_aerodromo_destino) destino on destino.icao_aerodromo_destino=aerodromos.icao
left join (select icao_aerodromo_origem, count(distinct(icao_aerodromo_destino)) as rotas_partida, count(icao_aerodromo_destino) as decolagens from vra
where situacao_voo='REALIZADO'
group by icao_aerodromo_origem) origem on origem.icao_aerodromo_origem=aerodromos.icao
left join (select icao_aerodromo_origem, air_cia.razao_social, count(*) as decolagens, RANK() OVER (partition by icao_aerodromo_origem  Order BY COUNT(*), icao_empresa_aerea DESC) as ranking from vra
left join air_cia on icao_empresa_aerea=icao
where vra.situacao_voo='REALIZADO'
group by icao_aerodromo_origem, icao_empresa_aerea, air_cia.razao_social) empresa on aerodromos.icao=empresa.icao_aerodromo_origem and ranking=1
where icao is not null;

In [0]:
%sql
select * from aeroportos_cia_mais_atuante;


name,icao,rotas_destino,pousos,rotas_partida,decolagens,cia_mais_atuante
Professor Urbano Ernesto Stumpf Airport,SBSJ,5,11,5,9,OMNI TÁXI AÉREO S.A.
Corumbá International Airport,SBCR,2,161,2,166,AZUL LINHAS AÉREAS BRASILEIRAS S/A
Rubem Berta International Airport,SBUG,2,164,2,165,AZUL LINHAS AÉREAS BRASILEIRAS S/A
Ted Stevens Anchorage International Airport,PANC,1,127,1,127,null
São Gonçalo do Amarante–Governador Aluízio Alves International Airport,SBSG,17,5892,15,5937,MODERN TRANSPORTE AEREO DE CARGA S.A
Londrina–Governador José Richa Airport,SBLO,8,2143,8,2138,null
Amílcar Cabral International Airport,GVAC,4,237,4,234,null
Leite Lopes Airport,SBRP,15,2992,15,2969,GOL LINHAS AÉREAS S.A. (EX- VRG LINHAS AÉREAS S.A.)
Presidente Castro Pinto International Airport,SBJP,8,3791,9,3845,null
Deputado Luís Eduardo Magalhães International Airport,SBSV,36,18781,37,18952,null


In [0]:
%sql
select * from cia_aerea_rota_mais_usada;

icao_empresa_aerea,razao_social,icao_aerodromo_origem,nome_aeroporto_origem,localizacao_aeroporto_origem,icao_aerodromo_destino,nome_aeroporto_destino,localizacao_aeroporto_destino
ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),SBJD,null,null,SBPA,Salgado Filho International Airport,"Porto Alegre, Rio Grande do Sul, Brazil"
ASO,AEROSUL TÁXI AÉREO LTDA (EX.: AUSTEN TÁXI AÉREO LTDA),SBPO,null,null,SSOG,Arapongas Airport,"Arapongas, Paraná, Brazil"
ASO,AEROSUL TÁXI AÉREO LTDA (EX.: AUSTEN TÁXI AÉREO LTDA),SBPO,null,null,SSOG,Arapongas Airport,"Arapongas, Paraná, Brazil"
AZU,AZUL LINHAS AÉREAS BRASILEIRAS S/A,MNMG,Augusto C. Sandino International Airport,"Managua, Nicaragua",SBEG,Eduardo Gomes International Airport,"Manaus, Amazonas, Brazil"
GLO,GOL LINHAS AÉREAS S.A. (EX- VRG LINHAS AÉREAS S.A.),SAEZ,Ministro Pistarini International Airport,"Buenos Aires, Argentina",SBGR,São Paulo–Guarulhos International Airport,"São Paulo, Brazil"
LTG,ABSA - AEROLINHAS BRASILEIRAS S.A.,KMIA,Miami International Airport,"Miami, Florida, United States",SUMU,Carrasco Gral. Cesáreo L. Berisso International Airport,"Montevideo, Uruguay"
MWM,MODERN TRANSPORTE AEREO DE CARGA S.A,SBBE,Val de Cans International Airport,"Belém, Pará, Brazil",SBEG,Eduardo Gomes International Airport,"Manaus, Amazonas, Brazil"
OMI,OMNI TÁXI AÉREO S.A.,SBCF,Tancredo Neves International Airport (Confins Int'l Airport),"Belo Horizonte, Minas Gerais, Brazil",SBJU,Juazeiro do Norte Airport,"Juazeiro do Norte, Ceará, Brazil"
PAM,MAP TRANSPORTES AÉREOS LTDA.,SBEG,Eduardo Gomes International Airport,"Manaus, Amazonas, Brazil",SWLB,Lábrea Airport,"Lábrea, Amazonas, Brazil"
PTB,PASSAREDO TRANSPORTES AÉREOS S.A.,SBGR,São Paulo–Guarulhos International Airport,"São Paulo, Brazil",SBRP,Leite Lopes Airport,"Ribeirão Preto, São Paulo, Brazil"


#### Extras:
  - Descrever qual estratégia você usaria para ingerir estes dados de forma incremental caso precise capturar esses dados a cada mes?
  
  As tabelas aerodromos e air_cia são tabelas cadatrais, ou seja, vão possuir os dados referentes aos aeroportos e companhias aereas repectivamente. Elas provavelmente não devem ter modificações frequentes, então acredito que a cada mês bastaria adicionar as novas companhias áereas e aeroportos que não estão na tabela ainda.
  
 Caso os dados possuam uma atualização frequente, acredito que o melhor cenário seria apagar e recriar essas duas tabelas todo mês.
 
 Em relação a tabela VRA que possui informações dos voo, podemos fazer a incremetação dos voo do mês passado, já que voos que já ocorreram não vão sofrer modificações, então pegariamos apenas os voo feitos no ultimos mês para adicionar a tabela principal.
  
  - Justifique em cada etapa sobre a escalabilidade da tecnologia utilizada.
  
  Escolhi utilizar o databricks possui o Spark possui uma melhor capacidade para lidar com alto volumes de dados, toda essa etapa poderia ter sido feita com o Pandas mas não seria uma tecnologia tão escalável como o Spark.
  
  Para acessar os dados da API utilizei o pacote requests de maneira simples, mas para um volume maior de dados as requisições poderiam ser feitas de maneira paralela.
  
  - Justifique as camadas utilizadas durante o processo de ingestão até a disponibilização dos dados.
  
  Separei os dados em 3 camadas.
  
  A primeira camada possui os dados brutos. Nela temos os arquivos disponibilidados e baixados via API.
  
  A segunda camada são as tabelas que foram retiradas dos arquivos, tratadas como spark e gravadas. (AIR_CIA, VRA e Aerodromos)
  
  A terceira camada seria as próprias views que já possuem certas regras de négocio envolvida. (aeroportos_cia_mais_atuante, cia_aerea_rota_mais_usada)